In [2]:
# 用窗口取数
import os
import pandas as pd

# 参数设置
window_size = 28
step_size = 14

# 读取数据
input_file = '../DATA/labeled_tactile_dataset_clean.csv'
output_file = '../DATA/labeled_window_dataset.csv'

df = pd.read_csv(input_file)

# 初始化存储窗口信息的列表
windowed_data = []

# 初始化 window_id
window_id = 0

# 循环以 step_size 步长遍历数据
for start in range(0, len(df) - window_size + 1, step_size):
    end = start + window_size
    window = df.iloc[start:end].copy()
    
    # 生成 window_id
    window['window_id'] = window_id
    window_id += 1  # 每次迭代增加 window_id
    
    # 判断窗口内 touch_type 是否全为 'NC'
    unique_touch_types = window['touch_type'].unique()
    if len(unique_touch_types) == 1 and unique_touch_types[0] == 'NC':
        window_touch_type = 'NC'
    else:
        # 如果不是全为 'NC'，取窗口内唯一的非 'NC' 的 touch_type
        non_nc_touch_types = [t for t in unique_touch_types if t != 'NC']
        window_touch_type = non_nc_touch_types[0] if non_nc_touch_types else 'NC'
    
    # 给窗口内所有行赋值 window_touch_type
    window['window_touch_type'] = window_touch_type
    
    # 添加窗口数据到列表
    windowed_data.append(window)

# 合并所有窗口数据
windowed_df = pd.concat(windowed_data, ignore_index=True)

# 删除已存在的文件
if os.path.exists(output_file):
    os.remove(output_file)

# 保存到新的CSV文件
windowed_df.to_csv(output_file, index=False)

print(f"{output_file} 文件已生成，总行数 = {len(windowed_df)}")


../DATA/labeled_window_dataset.csv 文件已生成，总行数 = 244692


In [6]:
dt = pd.read_csv('../DATA/labeled_window_dataset.csv')
dt
# dt[dt['window_touch_type']=='ST']['window_id'].nunique()
grouped = dt.groupby(['window_id', 'window_touch_type', 'touch_type']).size().reset_index(name='count')
# # 展示结果
print(grouped)
# dt[dt['window_id']==0]
# print(len(dt)/28)

      window_id window_touch_type touch_type  count
0             0                NC         NC     28
1             1                NC         NC     28
2             2                NC         NC     28
3             3                NC         NC     28
4             4                NC         NC     28
...         ...               ...        ...    ...
8767       8734                 G          G     28
8768       8735                 G          G     28
8769       8736                 G          G     28
8770       8737                 G          G     28
8771       8738                 G          G     28

[8772 rows x 4 columns]


In [7]:
print(dt.groupby('window_touch_type')['window_id'].nunique())

window_touch_type
DT     835
G     1627
NC    4238
P     1574
ST     465
Name: window_id, dtype: int64
